In [54]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [55]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('data/산업DB.v.0.3.csv')
docs = loader.load()
for i, d in enumerate(docs[:2]):
    print(i, d)

0 page_content='﻿KRX 업종명: IT 서비스
상세내용: 컴퓨터
특정 고객의 요구에 의하여 주문형 소프트웨어를 자문, 개발 및 공급
하는 산업활동을 말한다. 블록체인 기반 기술도 포함한다.
<예 시>
․주문형 응용 소프트웨어 제작
․주문형 컴퓨터 소프트웨어 분석․설계
<제 외>
․패키지형(범용성) 소프트웨어 개발 및 공급(582)
컴퓨터 시스템  통합  자문,  구축  및  관리업' metadata={'source': 'data/산업DB.v.0.3.csv', 'row': 0}
1 page_content='﻿KRX 업종명: IT 서비스
상세내용: 컴퓨터
컴퓨터
템을 기획, 설계하는 산업활동을 말한다. 이 사업체는 시스템 통합 서
비스를  위하여  구성  요소인  하드웨어와  소프트웨어의  개발,  판매,  시 스템 설치, 시스템 이용자 훈련을 부수적으로 수행할 수 있으며, 블록 체인 기반 기술도 포함한다.
<예 시>
․시스템 통합(SI) 구축 설계      ․시스템 통합 설계 자문
․근거리 통신망(LAN) 컴퓨터 시스템 통합 설계
․사무 자동화 컴퓨터 시스템 통합 설계
․정보 관리 컴퓨터 시스템 통합 설계
<제외>
․패키지형(범용성) 소프트웨어 개발 및 공급(582)' metadata={'source': 'data/산업DB.v.0.3.csv', 'row': 1}


In [56]:

len(docs)

1205

In [57]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [58]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [59]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2300, 
    chunk_overlap=230,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [60]:
len(chunks)

1205

In [61]:
# vector sotre에 저장
from langchain_pinecone import PineconeVectorStore

batch_size = 200
  # 한 번에 처리할 문서 수

for i in range(0, len(chunks), batch_size):
    batch_docs = chunks[i:i+batch_size]
    
    if i == 0:
        # 첫 번째 배치로 벡터 스토어 생성
        vector_store = PineconeVectorStore.from_documents(
            batch_docs,            # batch_size 수 만큼의 chunk
            embedding=embeddings,  # 임베딩 벡터로 변환
            index_name="lastproject",   # index 이름
            namespace="industry"    
        )
    else:
        # 이후 배치는 기존 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
        vector_store.add_documents(batch_docs)    
    
    print(f"배치 {i//batch_size + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 200개 문서 업로드
배치 2 완료: 200개 문서 업로드
배치 3 완료: 200개 문서 업로드
배치 4 완료: 200개 문서 업로드
배치 5 완료: 200개 문서 업로드
배치 6 완료: 200개 문서 업로드
배치 7 완료: 5개 문서 업로드
